In [30]:
from typing import List, Tuple
import numpy as np
import scipy as sp
import openfermion as of
import cirq
import qiskit
from cirq.contrib.qasm_import import circuit_from_qasm
from convert import cirq_pauli_sum_to_qiskit_pauli_op

In [19]:
# See Kirby paper
def upper_bound(H_norm, overlap, d, delta, delta_prime, Delta_prime, chi, zeta):
    term0 = delta_prime if delta_prime > Delta_prime else 0
    term1 = (1/overlap)*(1+6*H_norm/(delta_prime-delta))*chi
    term2 = (6*H_norm/overlap)*zeta
    term3 = (6*H_norm/overlap)*8*(1+0.5*np.pi*delta/H_norm)**(-2*d)
    out = term0+term1+term2+term3
    return out

In [20]:
def minimized_bound(H_norm, overlap_prime, d, Delta, Delta_prime, chi, zeta):
    def upper_bound_fn(x):
            return upper_bound(H_norm, overlap_prime, d, x[0], x[1], Delta_prime, chi, zeta)
        
    if Delta_prime > 0:
        x0 = (Delta_prime/2, Delta_prime)
    else:
        x0 = (Delta/2,Delta)
    
    
    # Minimize over values of delta and delta_prime:
    min_bound = sp.optimize.minimize(
        upper_bound_fn,
        method='COBYLA',
        x0=x0,
        constraints=(
            {'type': 'ineq', 'fun': lambda x:  x[1]-x[0]-chi/overlap_prime},
            {'type': 'ineq', 'fun': lambda x:  x[0]},
            {'type': 'ineq', 'fun': lambda x:  x[1]},
            {'type': 'ineq', 'fun': lambda x:  (Delta_prime-0.01 if Delta_prime>0.01 else Delta) - x[1]}
        )
    )
    
    if min_bound.fun < upper_bound_fn(x0):
        bound = min_bound.fun
    else:
        print('Minimization failed bound', min_bound.fun, 'replaced with default (initial) point')
        bound = upper_bound_fn(x0)
    return bound

In [23]:
def fill_h_and_s_matrices(
    vectors: List[np.ndarray],
    matrix: np.ndarray,
    verbose: bool = False,
) -> Tuple[np.ndarray, np.ndarray]:
    dim = len(vectors)
    h = np.zeros((dim, dim), dtype=np.complex128)
    s = np.zeros((dim, dim), dtype=np.complex128)

    for i in range(dim):
        for j in range(i, dim):
            if verbose:
                print(i, j)
            hij = vectors[i].conj().T @ matrix @ vectors[j]
            h[i, j] = hij

            if i != j:
                h[j, i] = np.conjugate(hij)

            sij = vectors[i].conj().T @ vectors[j]
            s[i, j] = sij
            if i != j:
                s[j, i] = np.conjugate(sij)
    return h, s

In [25]:
# Based on https://quantum.cloud.ibm.com/docs/en/tutorials/krylov-quantum-diagonalization.
# and Algorithm 1.1 of https://arxiv.org/abs/2110.07492.
def solve_regularized_gen_eig(
    h: np.ndarray,
    s: np.ndarray,
    threshold: float,
) -> float:
    if np.isclose(threshold, 0, atol=1e-10):
        h_reg = h
        s_reg = s
    else:
        s_vals, s_vecs = sp.linalg.eigh(s)
        s_vecs = s_vecs.T
        good_vecs = np.array(
            [vec for val, vec in zip(s_vals, s_vecs) if val > threshold]
        )
        h_reg = good_vecs.conj() @ h @ good_vecs.T
        s_reg = good_vecs.conj() @ s @ good_vecs.T
        return sp.linalg.eigh(h_reg, s_reg)[0][0]

In [29]:
l = 2
t = 1.0
u = 4.0
hamiltonian_fermi = of.hamiltonians.fermi_hubbard(
    l, l, t, u, spinless=True, periodic=False
)
hamiltonian = of.transforms.jordan_wigner(hamiltonian_fermi)
ham_cirq = of.transforms.qubit_operator_to_pauli_sum(hamiltonian)
ham_qiskit = cirq_pauli_sum_to_qiskit_pauli_op(ham_cirq)
matrix = ham_cirq.matrix()

evals_exact, evecs_exact = np.linalg.eigh(matrix)
ground_state = evecs_exact.T[0]
energy_exact = min(evals_exact)
print("Exact energy:", energy_exact)

bvec = ground_state + np.random.randn(ground_state.shape[0]) / 5.0
bvec /= np.linalg.norm(bvec)

overlap = np.abs(bvec.conj().T @ ground_state) ** 2
print("Overlap =", overlap)

Exact energy: -2.0
Overlap = 0.4424394476908869


## Get subspace matrices with U.

In [27]:
subspace_dimension = 40
threshold = 1e-4

In [16]:
dt = np.pi / (evals_exact[-1] - evals_exact[0])

vectors_u = []
for k in range(-2 * subspace_dimension, 2 * subspace_dimension + 1, 1):
    print(k)
    Uk = sp.linalg.expm(-1j * matrix * k * dt)
    vectors_u.append(Uk @ bvec)

-80
-79
-78
-77
-76
-75
-74
-73
-72
-71
-70
-69
-68
-67
-66
-65
-64
-63
-62
-61
-60
-59
-58
-57
-56
-55
-54
-53
-52
-51
-50
-49
-48
-47
-46
-45
-44
-43
-42
-41
-40
-39
-38
-37
-36
-35
-34
-33
-32
-31
-30
-29
-28
-27
-26
-25
-24
-23
-22
-21
-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/scipy/linalg/_matfuncs.py:373: RuntimeWarning: divide by zero encountered in matmul
  eAw = eAw @ eAw
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/scipy/linalg/_matfuncs.py:373: RuntimeWarning: overflow encountered in matmul
  eAw = eAw @ eAw
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/scipy/linalg/_matfuncs.py:373: RuntimeWarning: invalid value encountered in matmul
  eAw = eAw @ eAw


In [26]:
h_u, s_u = fill_h_and_s_matrices(vectors_u, matrix)

In [28]:
krylov_u_evals = []
for d in range(1, len(h_u)):
    krylov_energy = solve_regularized_gen_eig(h_u[:d, :d], s_u[:d, :d], threshold=threshold)
    print(krylov_energy)
    krylov_u_evals.append(krylov_energy)

1.6139733338391429
-1.2296091567941514
-1.6516412386245005
-1.8544004160518905
-1.9552752580836157
-1.9909287616116338
-1.987385028441333
-1.9960765868279273
-1.9947092424793915
-1.9926012726543365
-1.9974486084163843
-1.996654457501794
-1.9994038336565043
-1.9992063191393947
-1.9988257652003243
-2.0
-2.000000000000003
-2.0000000000000013
-2.0
-2.000000000000003
-2.000000000000002
-1.9999999999999998
-1.9999999999999998
-2.0000000000000004
-1.9999999999999993
-2.000000000000001
-1.9999999999999987
-2.0000000000000018
-1.9999999999999996
-2.0000000000000004
-1.9999999999999987
-1.999999999999999
-2.0000000000000004
-2.0000000000000018
-2.0000000000000027
-2.000000000000001
-1.9999999999999984
-1.9999999999999993
-1.9999999999999998
-2.0000000000000004
-1.9999999999999987
-1.9999999999999982
-1.9999999999999993
-2.0
-1.999999999999998
-2.0000000000000018
-2.0000000000000004
-1.9999999999999996
-1.9999999999999984
-1.9999999999999998
-2.0
-1.9999999999999993
-2.0000000000000013
-2.0
-1.99

/var/folders/62/vpq4vzfx3zlfyyr3mw8zdmrc0000gn/T/ipykernel_62566/215692311.py:17: RuntimeWarning: divide by zero encountered in matmul
  h_reg = good_vecs.conj() @ h @ good_vecs.T
/var/folders/62/vpq4vzfx3zlfyyr3mw8zdmrc0000gn/T/ipykernel_62566/215692311.py:17: RuntimeWarning: overflow encountered in matmul
  h_reg = good_vecs.conj() @ h @ good_vecs.T
/var/folders/62/vpq4vzfx3zlfyyr3mw8zdmrc0000gn/T/ipykernel_62566/215692311.py:17: RuntimeWarning: invalid value encountered in matmul
  h_reg = good_vecs.conj() @ h @ good_vecs.T
/var/folders/62/vpq4vzfx3zlfyyr3mw8zdmrc0000gn/T/ipykernel_62566/215692311.py:18: RuntimeWarning: divide by zero encountered in matmul
  s_reg = good_vecs.conj() @ s @ good_vecs.T
/var/folders/62/vpq4vzfx3zlfyyr3mw8zdmrc0000gn/T/ipykernel_62566/215692311.py:18: RuntimeWarning: overflow encountered in matmul
  s_reg = good_vecs.conj() @ s @ good_vecs.T
/var/folders/62/vpq4vzfx3zlfyyr3mw8zdmrc0000gn/T/ipykernel_62566/215692311.py:18: RuntimeWarning: invalid value e

-2.0000000000000004
-1.999999999999999
-1.9999999999999978
-2.000000000000001
-2.0
-2.0000000000000004
-2.000000000000001
-1.9999999999999984
-1.999999999999999
-1.9999999999999996
-1.9999999999999998
-1.999999999999999
-2.000000000000001
-2.0000000000000013
-2.000000000000001
-2.0000000000000027
-2.000000000000001


## Get subspace matrices with Trotter.

In [31]:
ntrotter_values = [1, 2, 4, 8, 16, 32]
ngates_twoq = []
ngates_oneq = []
depths = []
all_krylov_u_trotter_evals = []
all_trotter_h = []
all_trotter_s = []

for ntrotter in ntrotter_values:
    print("On ntrotter =", ntrotter)
    trotter_operation = qiskit.circuit.library.PauliEvolutionGate(
        ham_qiskit,
        time=dt,
        synthesis=qiskit.synthesis.LieTrotter(reps=ntrotter)
    )

    trotter_circuit = qiskit.QuantumCircuit.from_instructions(
        [[trotter_operation]],
        qubits=qiskit.QuantumRegister(trotter_operation.num_qubits),
    )
    trotter_circuit = qiskit.transpile(
        trotter_circuit, basis_gates=["u3", "cx"]
    )  # TODO: Compile to a target backend, e.g. IBM Fez.
    opcounts = trotter_circuit.count_ops()
    ngates_twoq.append(opcounts.get("cx"))
    ngates_oneq.append(opcounts.get("u3"))
    depths.append(trotter_circuit.depth())

    qasm = qiskit.qasm2.dumps(trotter_circuit)
    trotter_circuit_cirq = circuit_from_qasm(qasm.replace("qregless", "q"))

    qubits = sorted(trotter_circuit_cirq.all_qubits())
    Utrotter = cirq.unitary(trotter_circuit_cirq)

    vectors_u_trotter = []
    for k in range(-2 * subspace_dimension, 2 * subspace_dimension + 1, 1):
        # print(k)
        Uk = np.linalg.matrix_power(Utrotter, k)
        vectors_u_trotter.append(Uk @ bvec)
    
    h, s = fill_h_and_s_matrices(vectors_u_trotter, matrix)
    krylov_u_trotter_evals = []
    for d in range(1, len(h)):
        krylov_energy = solve_regularized_gen_eig(h[:d, :d], s[:d, :d], threshold=threshold)
        krylov_u_trotter_evals.append(krylov_energy)
    
    all_krylov_u_trotter_evals.append(krylov_u_trotter_evals)
    all_trotter_h.append(h)
    all_trotter_s.append(s)

On ntrotter = 1


/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/benjamindalfavero/.venv/compare/lib/python3.13/site-packages/scipy/sparse/linalg/_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)


TranspilerError: "HighLevelSynthesis: number of operation's qubits (4) does not match the circuit size (0)"